In [ ]:
%matplotlib qt
import h5py, tifffile, time, os, gc
import numpy as np
from scipy.ndimage import fourier_shift
from scipy.optimize import minimize

import matplotlib.pyplot as plt
#from multiprocessing import set_start_method
#set_start_method("spawn")
import multiprocessing as mp
#from multiprocessing import get_context

from functools import partial

import imagej
ij = imagej.init('/home/xiao/software/Fiji.app', headless=False)
ijui = ij.ui()
ijui.showUI()

from jnius import autoclass
wm = autoclass('ij.WindowManager')
ImagePlusClass = autoclass('ij.ImagePlus')

from TXM_Sandbox.TXM_Sandbox.utils.xanes_math import multi_resolutin_tv_reg

In [ ]:
fn = '/home/xiao/data/xanes_align_test/2D_trial_reg_multipos_2D_xanes_scan2_id_61201_repeat_00_pos_02_2020-09-21-08-39-47.h5'
pair_id = 1
with h5py.File(fn, 'r') as f:
    img = f['/trial_registration/trial_reg_results/{0}/trial_reg_img{0}'.format(str(pair_id).zfill(3))][:]
    fixed_img = f['/trial_registration/trial_reg_results/{0}/trial_reg_fixed{0}'.format(str(pair_id).zfill(3))][:]
    
ijui.show(ij.py.to_java(img-fixed_img))

diff_img = np.zeros(fixed_img.shape)

# ***brute force approach***

In [ ]:
def tv_pixel(fixed_img, img, y_shift, x_shift):
    diff_img = fixed_img - np.roll(img, [y_shift, x_shift], axis=[0, 1])
    tv1 = (np.abs(np.diff(diff_img, axis=0, prepend=1))+np.abs(np.diff(diff_img, axis=1, prepend=1))).sum()
    tv2 = ((np.diff(diff_img, axis=0, prepend=1))**2+(np.diff(diff_img, axis=1, prepend=1))**2).sum()
    return tv1, tv2 

def tv_subpixel(fixed_img, img, y_shift, x_shift):
    diff_img = fixed_img - np.real(np.fft.ifftn(fourier_shift(np.fft.fftn(img), [y_shift, x_shift])))
    tv1 = (np.abs(np.diff(diff_img, axis=0, prepend=1))+np.abs(np.diff(diff_img, axis=1, prepend=1))).sum()
    tv2 = ((np.diff(diff_img, axis=0, prepend=1))**2+(np.diff(diff_img, axis=1, prepend=1))**2).sum()
    return tv1, tv2 

In [ ]:
w = 40
h = 40

"""
print(time.asctime())
tv = []
for ii in np.int32(np.arange(w*h)):
    tv.append(tv_pixel(fixed_img, img, (ii//w-int(h/2)), (ii%w-int(w/2))))
    
#tv = np.array(tv)

tv1_pxl = np.ndarray([h, w], dtype=np.float32)
tv2_pxl = np.ndarray([h, w], dtype=np.float32)
for ii in range(w*h):
    tv1_pxl[ii//w, ii%w] = tv[ii][0]
    tv2_pxl[ii//w, ii%w] = tv[ii][1]
print(time.asctime())


tv1_pxl_id = tv1_pxl.argmin()
tv2_pxl_id = tv2_pxl.argmin()
y_pxl_shift = tv1_pxl_id//w-int(h/2)
x_pxl_shift = tv1_pxl_id%w-int(w/2)


print(time.asctime())
w = 20
h = 20
step = 0.2
tv = []

for ii in np.int32(np.arange(w*h)):
    tv.append(tv_subpixel(fixed_img, img, (ii//w-int(h/2))*step+y_pxl_shift, (ii%w-int(w/2))*step+x_pxl_shift))
    
tv1_subpxl = np.ndarray([h, w], dtype=np.float32)
tv2_subpxl = np.ndarray([h, w], dtype=np.float32)
for ii in range(w*h):
    tv1_subpxl[ii//w, ii%w] = tv[ii][0]
    tv2_subpxl[ii//w, ii%w] = tv[ii][1]
print(time.asctime())

"""
print(time.asctime())
n_cpu = os.cpu_count()
#with get_context('spawn').Pool(n_cpu-1) as pool:
with mp.Pool(int(n_cpu/2)) as pool:
    rlt = pool.starmap(tv_pixel, [(fixed_img, img, (ii//w-int(h/2)), (ii%w-int(w/2))) for ii in np.int32(np.arange(w*h))])
pool.join()
pool.close()
print(time.asctime())

tv1_pxl = np.ndarray([h, w], dtype=np.float32)
tv2_pxl = np.ndarray([h, w], dtype=np.float32)
for ii in range(w*h):
    tv1_pxl[ii//w, ii%w] = rlt[ii][0]
    tv2_pxl[ii//w, ii%w] = rlt[ii][1]
del(rlt)
gc.collect()
print(time.asctime())

tv1_pxl_id = tv1_pxl.argmin()
tv2_pxl_id = tv2_pxl.argmin()
y_pxl_shift = tv1_pxl_id//w-int(h/2)
x_pxl_shift = tv1_pxl_id%w-int(w/2)

w = 20
h = 20
step = 0.2

print(time.asctime())
n_cpu = os.cpu_count()
#with get_context('spawn').Pool(n_cpu-1) as pool:
with mp.Pool(int(n_cpu/2)) as pool:
    rlt = pool.starmap(tv_subpixel, [(fixed_img, img, (ii//w-int(h/2))*step+y_pxl_shift, \
                                      (ii%w-int(w/2))*step+x_pxl_shift) for ii in np.int32(np.arange(w*h))])
pool.join()
pool.close()
print(time.asctime())

tv1_subpxl = np.ndarray([h, w], dtype=np.float32)
tv2_subpxl = np.ndarray([h, w], dtype=np.float32)
for ii in range(w*h):
    tv1_subpxl[ii//w, ii%w] = rlt[ii][0]
    tv2_subpxl[ii//w, ii%w] = rlt[ii][1]
del(rlt)
gc.collect()

print(time.asctime())

In [ ]:
### correct shift: y:-8.5, x:10
tv1_subpxl_id = tv1_subpxl.argmin()
tv2_subpxl_id = tv2_subpxl.argmin()
y_subpxl_shift = (tv1_subpxl_id//w-int(h/2))*step
x_subpxl_shift = (tv1_subpxl_id%w-int(w/2))*step
print(tv1_subpxl_id)
print(tv2_subpxl_id)

plt.figure(1)
plt.plot(tv1_pxl.flatten())
#plt.figure(2)
#plt.plot(tv2_pxl.flatten())
plt.figure(3)
plt.plot(tv1_subpxl.flatten())
#plt.figure(4)
#plt.plot(tv2_subpxl.flatten())

ijui.show(ij.py.to_java(fixed_img - np.real(np.fft.ifftn(
    fourier_shift(np.fft.fftn(img), [y_pxl_shift+y_subpxl_shift, x_pxl_shift+x_subpxl_shift])))))
wm.getCurrentWindow().setTitle('subpixel')
ijui.show(ij.py.to_java(fixed_img - np.real(np.fft.ifftn(
    fourier_shift(np.fft.fftn(img), [y_pxl_shift, x_pxl_shift])))))
wm.getCurrentWindow().setTitle('intpixel')
#ijui.show(ij.py.to_java(fixed_img - np.roll(img, [tv2_pxl_id//w-int(h/2), tv2_pxl_id%w-int(w/2)], axis=[0, 1])))
#ijui.show(ij.py.to_java(fixed_img - np.roll(img, [-9, 10], axis=[0, 1])))

In [ ]:
def tv_opt(shift, fixed_img, img):
    diff_img = fixed_img - np.real(np.fft.ifftn(fourier_shift(np.fft.fftn(img), shift)))
    tv1 = (np.abs(np.diff(diff_img, axis=0, prepend=1))+np.abs(np.diff(diff_img, axis=1, prepend=1))).sum()
    #tv2 = ((np.diff(diff_img, axis=0, prepend=1))**2+(np.diff(diff_img, axis=1, prepend=1))**2).sum()
    return tv1

res = minimize(tv_opt, [0, 0], args=(fixed_img, img), method='CG')
"""
n_cpu = os.cpu_count()
print(time.asctime())
with mp.Pool(n_cpu-1) as pool:
    rlt = pool.starmap(tv_opt, [((ii//w-int(h/2)), (ii%w-int(w/2))) for ii in range(w*h)])

tv1_pxl = np.ndarray([h, w], dtype=np.float32)
tv2_pxl = np.ndarray([h, w], dtype=np.float32)
for ii in range(w*h):
    tv1_pxl[ii%w, ii//w] = rlt[ii][0]
    tv2_pxl[ii%w, ii//w] = rlt[ii][0]
"""

In [ ]:
print(res)

In [ ]:
plt.figure(1)
plt.imshow(np.array(tv1).reshape(20,20))

plt.figure(2)
plt.imshow(np.array(tv2).reshape(20,20))

print(np.array(tv1).argmin())
print(tv1[290])
print(np.array(tv2).argmin())
print(tv2[290])

plt.figure(3)
plt.plot(tv1)

plt.figure(4)
plt.plot(tv2)

In [ ]:
diff_img[:] = (np.real(np.fft.ifftn(fourier_shift(np.fft.fftn(img), [(12-int(w/2))*0.5, (10-int(w/2))*0.5])))-fixed_img)[:]
ijui.show(ij.py.to_java(diff_img))

# ***separate line optimization***

In [ ]:
def tv_pixel(fixed_img, img, y_shift, x_shift):
    diff_img = fixed_img - np.roll(img, [y_shift, x_shift], axis=[0, 1])
    tv1 = (np.abs(np.diff(diff_img, axis=0, prepend=1))+np.abs(np.diff(diff_img, axis=1, prepend=1))).sum()
    tv2 = ((np.diff(diff_img, axis=0, prepend=1))**2+(np.diff(diff_img, axis=1, prepend=1))**2).sum()
    fn = f'/media/Disk2/data/Zhi/debug/shifted_img_{str(y_shift).zfill(2)}_{str(x_shift).zfill(2)}.tiff'
    tifffile.imsave(fn, diff_img.astype(np.float32))
    return tv1, tv2 
    #return tv1

def tv_pixel_y(fixed_img, img, y_shift):
    diff_img = fixed_img - np.roll(img, y_shift, axis=0)
    tv1 = np.abs(np.diff(diff_img, axis=0, prepend=1)).sum()
    tv2 = ((np.diff(diff_img, axis=0, prepend=1))**2).sum()
    fn = f'/media/Disk2/data/Zhi/debug/shifted_img_y_{str(y_shift).zfill(2)}.tiff'
    tifffile.imsave(fn, diff_img.astype(np.float32))
    return tv1, tv2 
    #return tv1

def tv_pixel_x(fixed_img, img, y_shift, x_shift):
    diff_img = fixed_img - np.roll(img, [y_shift, x_shift], axis=[0, 1])
    tv1 = np.abs(np.diff(diff_img, axis=1, prepend=1)).sum()
    tv2 = ((np.diff(diff_img, axis=1, prepend=1))**2).sum()
    fn = f'/media/Disk2/data/Zhi/debug/shifted_img_x_{str(x_shift).zfill(2)}.tiff'
    tifffile.imsave(fn, diff_img.astype(np.float32))
    return tv1, tv2 
    #return tv1

def tv_subpixel(fixed_img, img, y_shift, x_shift):
    diff_img = fixed_img - np.real(np.fft.ifftn(fourier_shift(np.fft.fftn(img), [y_shift, x_shift])))
    tv1 = (np.abs(np.diff(diff_img, axis=0, prepend=1))+np.abs(np.diff(diff_img, axis=1, prepend=1))).sum()
    tv2 = ((np.diff(diff_img, axis=0, prepend=1))**2+(np.diff(diff_img, axis=1, prepend=1))**2).sum()
    return tv1, tv2 
    #return tv1

def tv_opt_v2(fixed_img, img, h, w):
    tv1_y = []
    tv1_x = []
    
    
    for ii in range(h):
        tv1_y.append(tv_pixel_y(fixed_img, img, ii-int(h/2))[0])
    tv1_y_id = np.array(tv1_y).argmin()
    tv1_y_shift = tv1_y_id-int(h/2)
    for ii in range(w):
        tv1_x.append(tv_pixel_x(fixed_img, img, tv1_y_shift, ii-int(w/2))[0])
    tv1_x_id = np.array(tv1_x).argmin()
    tv1_x_shift = tv1_x_id-int(w/2)
    
    
    """
    for ii in range(h):
        tv1_y.append(tv_subpixel(fixed_img, img, (ii-int(h/2))*0.5, 0)[0])
    tv1_y_id = np.array(tv1_y).argmin()
    tv1_y_shift = (tv1_y_id-int(h/2))*0.5
    for ii in range(w):
        tv1_x.append(tv_subpixel(fixed_img, img, tv1_y_shift, (ii-int(w/2))*0.5)[0])
    tv1_x_id = np.array(tv1_x).argmin()
    tv1_x_shift = (tv1_x_id-int(w/2))*0.5
    """
    
    tv1_y_sub = []
    tv1_x_sub = []
    for ii in range(20):
        tv1_y_sub.append(tv_subpixel(fixed_img, img, (ii-10)*0.2+tv1_y_shift, tv1_x_shift)[0])
    tv1_sub_y_id = np.array(tv1_y_sub).argmin()
    tv1_sub_y_shift = (tv1_sub_y_id-10)*0.2+tv1_y_shift
    for ii in range(20):
        tv1_x_sub.append(tv_subpixel(fixed_img, img, tv1_sub_y_shift, (ii-10)*0.2+tv1_x_shift)[0])
    tv1_sub_x_id = np.array(tv1_x_sub).argmin()
    tv1_sub_x_shift = (tv1_sub_x_id-10)*0.2+tv1_x_shift
    
    return tv1_y, tv1_x, tv1_y_sub, tv1_x_sub, tv1_sub_y_shift, tv1_sub_x_shift

In [ ]:
print(time.asctime())
tv1_y, tv1_x, tv1_y_sub, tv1_x_sub, tv1_sub_y_shift, tv1_sub_x_shift = tv_opt_v2(fixed_img, img, 40, 40)
print(time.asctime())

diff_img = (np.real(np.fft.ifftn(fourier_shift(np.fft.fftn(img), [tv1_sub_y_shift, tv1_sub_x_shift])))-fixed_img)
ijui.show(ij.py.to_java(diff_img))

In [ ]:
print(np.array(tv1_x).argmin())

In [ ]:
plt.figure(11)
plt.plot(tv1_y)
plt.figure(12)
plt.plot(tv1_x)
plt.figure(13)
plt.plot(tv1_y_sub)
plt.figure(14)
plt.plot(tv1_x_sub)

# ***multi-resolution approach***

In [ ]:
def tv_pixel(fixed_img, img, y_shift, x_shift):
    diff_img = fixed_img - np.roll(img, [y_shift, x_shift], axis=[0, 1])
    tv1 = (np.abs(np.diff(diff_img, axis=0, prepend=1))+np.abs(np.diff(diff_img, axis=1, prepend=1))).sum()
    tv2 = ((np.diff(diff_img, axis=0, prepend=1))**2+(np.diff(diff_img, axis=1, prepend=1))**2).sum()
    return tv1, tv2 

def tv_subpixel(fixed_img, img, y_shift, x_shift):
    diff_img = fixed_img - np.real(np.fft.ifftn(fourier_shift(np.fft.fftn(img), [y_shift, x_shift])))
    tv1 = (np.abs(np.diff(diff_img, axis=0, prepend=1))+np.abs(np.diff(diff_img, axis=1, prepend=1))).sum()
    tv2 = ((np.diff(diff_img, axis=0, prepend=1))**2+(np.diff(diff_img, axis=1, prepend=1))**2).sum()
    return tv1, tv2 

In [ ]:
### level 1
w = 20
h = 20
print(time.asctime())
n_cpu = os.cpu_count()
#with get_context('spawn').Pool(n_cpu-1) as pool:
with mp.Pool(int(n_cpu/2)) as pool:
    rlt = pool.starmap(tv_pixel, [(fixed_img, img, 4*(ii//w-int(h/2)), 4*(ii%w-int(w/2))) for ii in np.int32(np.arange(w*h))])
pool.join()
pool.close()

tv1_pxl_lev1 = np.ndarray([h, w], dtype=np.float32)
tv2_pxl_lev1 = np.ndarray([h, w], dtype=np.float32)
for ii in range(w*h):
    tv1_pxl_lev1[ii//w, ii%w] = rlt[ii][0]
    tv2_pxl_lev1[ii//w, ii%w] = rlt[ii][1]
del(rlt)
gc.collect()
print(time.asctime())

tv1_pxl_id_lev1 = tv1_pxl_lev1.argmin()
tv2_pxl_id_lev1 = tv2_pxl_lev1.argmin()
y_pxl_shift_lev1 = 4*(tv1_pxl_id_lev1//w-int(h/2))
x_pxl_shift_lev1 = 4*(tv1_pxl_id_lev1%w-int(w/2))
print(tv1_pxl_id_lev1, y_pxl_shift_lev1, x_pxl_shift_lev1)

### level 2
w = 10
h = 10
print(time.asctime())
n_cpu = os.cpu_count()
with mp.Pool(int(n_cpu/2)) as pool:
    rlt = pool.starmap(tv_pixel, [(fixed_img, img, 2*(ii//w-int(h/2))+y_pxl_shift_lev1, 
                                   2*(ii%w-int(w/2))+x_pxl_shift_lev1) for ii in np.int32(np.arange(w*h))])
    
#with mp.Pool(int(n_cpu/2)) as pool:
#    rlt = pool.starmap(tv_pixel, [(fixed_img, img, (ii//w-int(h/2)), (ii%w-int(w/2))) for ii in np.int32(np.arange(w*h))])
pool.join()
pool.close()
tv1_pxl_lev2 = np.ndarray([h, w], dtype=np.float32)
tv2_pxl_lev2 = np.ndarray([h, w], dtype=np.float32)
for ii in range(w*h):
    tv1_pxl_lev2[ii//w, ii%w] = rlt[ii][0]
    tv2_pxl_lev2[ii//w, ii%w] = rlt[ii][1]
del(rlt)
gc.collect()
print(time.asctime())

tv1_pxl_id_lev2 = tv1_pxl_lev2.argmin()
tv2_pxl_id_lev2 = tv2_pxl_lev2.argmin()
y_pxl_shift_lev2 = 2*(tv1_pxl_id_lev2//w-int(h/2))
x_pxl_shift_lev2 = 2*(tv1_pxl_id_lev2%w-int(w/2))
print(tv1_pxl_id_lev2, y_pxl_shift_lev2, x_pxl_shift_lev2)

### level 3
w = 10
h = 10
print(time.asctime())
n_cpu = os.cpu_count()
with mp.Pool(int(n_cpu/2)) as pool:
    rlt = pool.starmap(tv_pixel, [(fixed_img, img, (ii//w-int(h/2))+y_pxl_shift_lev1+y_pxl_shift_lev2, 
                                   (ii%w-int(w/2))+x_pxl_shift_lev1+x_pxl_shift_lev2) for ii in np.int32(np.arange(w*h))])

pool.join()
pool.close()
tv1_pxl_lev3 = np.ndarray([h, w], dtype=np.float32)
tv2_pxl_lev3 = np.ndarray([h, w], dtype=np.float32)
for ii in range(w*h):
    tv1_pxl_lev3[ii//w, ii%w] = rlt[ii][0]
    tv2_pxl_lev3[ii//w, ii%w] = rlt[ii][1]
del(rlt)
gc.collect()
print(time.asctime())

tv1_pxl_id_lev3 = tv1_pxl_lev3.argmin()
tv2_pxl_id_lev3 = tv2_pxl_lev3.argmin()
y_pxl_shift_lev3 = (tv1_pxl_id_lev3//w-int(h/2))
x_pxl_shift_lev3 = (tv1_pxl_id_lev3%w-int(w/2))
print(tv1_pxl_id_lev3, y_pxl_shift_lev3, x_pxl_shift_lev3)

### level 4
w = 10
h = 10
step = 0.2

print(time.asctime())
n_cpu = os.cpu_count()
#with get_context('spawn').Pool(n_cpu-1) as pool:
with mp.Pool(int(n_cpu/2)) as pool:
    rlt = pool.starmap(tv_subpixel, [(fixed_img, img, (ii//w-int(h/2))*step+y_pxl_shift_lev1+y_pxl_shift_lev2+y_pxl_shift_lev3, \
                                      (ii%w-int(w/2))*step+x_pxl_shift_lev1+x_pxl_shift_lev2+x_pxl_shift_lev3) for ii in np.int32(np.arange(w*h))])
pool.join()
pool.close()

tv1_subpxl = np.ndarray([h, w], dtype=np.float32)
tv2_subpxl = np.ndarray([h, w], dtype=np.float32)
for ii in range(w*h):
    tv1_subpxl[ii//w, ii%w] = rlt[ii][0]
    tv2_subpxl[ii//w, ii%w] = rlt[ii][1]
del(rlt)
gc.collect()
print(time.asctime())

tv1_subpxl_id = tv1_subpxl.argmin()
tv2_subpxl_id = tv2_subpxl.argmin()
y_subpxl_shift = (tv1_subpxl_id//w-int(h/2))*step
x_subpxl_shift = (tv1_subpxl_id%w-int(w/2))*step
print(tv1_subpxl_id, y_subpxl_shift, x_subpxl_shift)

In [ ]:
plt.figure(1)
plt.plot(tv1_pxl_lev1.flatten())
plt.figure(2)
plt.plot(tv1_pxl_lev2.flatten())
plt.figure(3)
plt.plot(tv1_pxl_lev3.flatten())
plt.figure(4)
plt.plot(tv1_subpxl.flatten())


ijui.show(ij.py.to_java(fixed_img - np.real(np.fft.ifftn(
    fourier_shift(np.fft.fftn(img), [y_pxl_shift_lev1, 
                                     x_pxl_shift_lev1])))))
wm.getCurrentWindow().setTitle('level 1')

ijui.show(ij.py.to_java(fixed_img - np.real(np.fft.ifftn(
    fourier_shift(np.fft.fftn(img), [y_pxl_shift_lev1+y_pxl_shift_lev2, 
                                     x_pxl_shift_lev1+x_pxl_shift_lev2])))))
wm.getCurrentWindow().setTitle('level 2')

ijui.show(ij.py.to_java(fixed_img - np.real(np.fft.ifftn(
    fourier_shift(np.fft.fftn(img), [y_pxl_shift_lev1+y_pxl_shift_lev2+y_pxl_shift_lev3, 
                                     x_pxl_shift_lev1+x_pxl_shift_lev2+x_pxl_shift_lev3])))))
wm.getCurrentWindow().setTitle('level 3')     

ijui.show(ij.py.to_java(fixed_img - np.real(np.fft.ifftn(
    fourier_shift(np.fft.fftn(img), [y_pxl_shift_lev1+y_pxl_shift_lev2+y_pxl_shift_lev3+y_subpxl_shift, 
                                     x_pxl_shift_lev1+x_pxl_shift_lev2+x_pxl_shift_lev3+x_subpxl_shift])))))
wm.getCurrentWindow().setTitle('level 4')

In [ ]:
cwi = wm.getCurrentWindow()

In [ ]:
import numpy as np
a = np.ones([5,2])
print(a[:0, 0].sum(axis=0))

In [ ]:
test = tifffile.imread('/run/media/xiao/Data/data/WeijiangXue_2020/54307_org_raw_projection_#50.tif')

# ***alignment class***

In [ ]:
print(time.asctime())
#for ii in range(5):
#    tv1_pxl, tv1_pxl_id, shift_list, shift = multi_resolutin_tv_reg(fixed_img, img, levs=4, wz=10, step=0.2)
tv1_pxl, tv1_pxl_id, shift_list, shift = multi_resolutin_tv_reg(fixed_img, img, levs=4, wz=10, step=0.2)
print(time.asctime())

In [ ]:
ijui.show(ij.py.to_java(fixed_img - np.real(np.fft.ifftn(
    fourier_shift(np.fft.fftn(img), [shift[:1, 0].sum(), 
                                     shift[:1, 1].sum()])))))
wm.getCurrentWindow().setTitle('level 1')

ijui.show(ij.py.to_java(fixed_img - np.real(np.fft.ifftn(
    fourier_shift(np.fft.fftn(img), [shift[:2, 0].sum(), 
                                     shift[:2, 1].sum()])))))
wm.getCurrentWindow().setTitle('level 2')

ijui.show(ij.py.to_java(fixed_img - np.real(np.fft.ifftn(
    fourier_shift(np.fft.fftn(img), [shift[:3, 0].sum(), 
                                     shift[:3, 1].sum()])))))
wm.getCurrentWindow().setTitle('level 3')     

ijui.show(ij.py.to_java(fixed_img - np.real(np.fft.ifftn(
    fourier_shift(np.fft.fftn(img), [shift[:4, 0].sum(), 
                                     shift[:4, 1].sum()])))))
wm.getCurrentWindow().setTitle('level 4')

ijui.show(ij.py.to_java(fixed_img - np.real(np.fft.ifftn(
    fourier_shift(np.fft.fftn(img), [shift[:5, 0].sum(), 
                                     shift[:5, 1].sum()])))))
wm.getCurrentWindow().setTitle('level 5')

In [ ]:
plt.figure(10)
plt.plot(tv1_pxl[3, :, :].flatten())

In [ ]:
print(tv1_pxl.shape)

In [ ]:
print(tv1_pxl_id)

In [13]:
import time, os, gc

import numpy as np, h5py
import multiprocess as mp
from scipy.ndimage import fourier_shift

fn = '/run/media/xiao/Data/data/2D_xanes/2D_trial_reg_xanes_scan2_id_54311_2020-07-01-11-20-34.h5'
pair_id = 1
with h5py.File(fn, 'r') as f:
    img = f['/trial_registration/trial_reg_results/{0}/trial_reg_img{0}'.format(str(pair_id).zfill(3))][:]
    fixed_img = f['/trial_registration/trial_reg_results/{0}/trial_reg_fixed{0}'.format(str(pair_id).zfill(3))][:]

def multi_resolutin_tv_reg1(fixed_img, img, levs=4, wz=10, step=0.2):
    def _tv_l1_pixel(fixed_img, img, y_shift, x_shift):
        diff_img = fixed_img - np.roll(img, [y_shift, x_shift], axis=[0, 1])
        tv1 = (np.abs(np.diff(diff_img, axis=0, prepend=1))+np.abs(np.diff(diff_img, axis=1, prepend=1))).sum()
        return tv1
    
    
    def _tv_l1_subpixel(fixed_img, img, y_shift, x_shift):
        diff_img = fixed_img - np.real(np.fft.ifftn(fourier_shift(np.fft.fftn(img), [y_shift, x_shift])))
        tv1 = (np.abs(np.diff(diff_img, axis=0, prepend=1))+np.abs(np.diff(diff_img, axis=1, prepend=1))).sum()
        return tv1

    sch_config = {}
    sch_config[levs-1] = {'wz':20, 'step':step}
    # tv1_pxl = np.ndarray([levs, wz, wz], dtype=np.float32)
    tv1_pxl = {}
    tv1_pxl_id = np.zeros(levs, dtype=np.int16)
    shift = np.zeros([levs, 2], dtype=np.float32)
    
    for ii in np.int32(np.arange(levs-1)):
        sch_config[levs-2-ii] = {'wz':wz, 'step':2**ii}
        
    # print(tv1_pxl.shape)
    
    n_cpu = os.cpu_count()
    

    for ii in range(levs-1): 
        print(ii)
        w = sch_config[ii]['wz']
        step = int(sch_config[ii]['step'])
        print(0.1)
        with mp.Pool(int(n_cpu-1)) as pool:
        # with mp.Pool(1) as pool:
            rlt = pool.starmap(_tv_l1_pixel, [(fixed_img, img, int(step*(jj//w-int(w/2))+shift[:ii, 0].sum()), 
                                                    int(step*(jj%w-int(w/2))+shift[:ii, 1].sum())) for jj in np.int32(np.arange(w**2))])
        print(0.2)
        pool.join()
        pool.close()
        
        print(0.3)
        tem = np.ndarray([w, w], dtype=np.float32)
        for kk in range(w**2):
            tem[kk//w, kk%w] = rlt[kk]
        del(rlt)
        gc.collect()
        
        tv1_pxl[ii] = np.array(tem)
        tv1_pxl_id[ii] = tv1_pxl[ii].argmin()
        shift[ii, 0] = step*(tv1_pxl_id[ii]//w-int(w/2))
        shift[ii, 1] = step*(tv1_pxl_id[ii]%w-int(w/2))
        print(ii)
        print(time.asctime())
        
    # print(tv1_pxl.shape)

    
    print(levs-1)
    w = sch_config[levs-1]['wz']
    step = sch_config[levs-1]['step']
    with mp.Pool(int(n_cpu-1)) as pool:
        rlt = pool.starmap(_tv_l1_subpixel, [(fixed_img, img, step*(jj//w-int(w/2))+shift[:(levs-1), 0].sum(), 
                                                    step*(jj%w-int(w/2))+shift[:(levs-1), 1].sum()) for jj in np.int32(np.arange(w**2))])
    pool.close() 
    pool.join()
    
    print(0.3)
    tem = np.ndarray([w, w], dtype=np.float32)
    for kk in range(w**2):
        tem[kk//w, kk%w] = rlt[kk]
    del(rlt)
    gc.collect()
    
    tv1_pxl[levs-1] = np.array(tem)
    tv1_pxl_id[levs-1] = tv1_pxl[levs-1].argmin()
    shift[levs-1, 0] = step*(tv1_pxl_id[levs-1]//w-int(w/2))
    shift[levs-1, 1] = step*(tv1_pxl_id[levs-1]%w-int(w/2))
    
    print(levs-1)
    # print(tv1_pxl.shape)
        
    return tv1_pxl, tv1_pxl_id, shift, shift.sum(axis=0)

In [14]:
print(time.asctime())
#for ii in range(5):
#    tv1_pxl, tv1_pxl_id, shift_list, shift = multi_resolutin_tv_reg(fixed_img, img, levs=4, wz=10, step=0.2)
tv1_pxl, tv1_pxl_id, shift_list, shift = multi_resolutin_tv_reg1(fixed_img, img, levs=4, wz=10, sp_wz=8, step=0.5)
print(time.asctime())

Sun Oct  4 16:47:48 2020
0
0.1
0.2
0.3
0
Sun Oct  4 16:47:52 2020
1
0.1
0.2
0.3
1
Sun Oct  4 16:47:56 2020
2
0.1
0.2
0.3
2
Sun Oct  4 16:48:00 2020
3
0.3
3
Sun Oct  4 16:48:45 2020


In [ ]:
shift = np.zeros([2, 2], dtype=np.float32)
print(shift[:1, 0])

In [7]:
import time, os, gc

import numpy as np, h5py
import multiprocess as mp
from concurrent.futures import TimeoutError
from pebble import ProcessPool, ProcessExpired
from functools import partial
from scipy.ndimage import fourier_shift

fn = '/run/media/xiao/Data/data/2D_xanes/2D_trial_reg_xanes_scan2_id_54311_2020-07-01-11-20-34.h5'
pair_id = 1
with h5py.File(fn, 'r') as f:
    img = f['/trial_registration/trial_reg_results/{0}/trial_reg_img{0}'.format(str(pair_id).zfill(3))][:]
    fixed_img = f['/trial_registration/trial_reg_results/{0}/trial_reg_fixed{0}'.format(str(pair_id).zfill(3))][:]

def tv_l1_pixel2(fixed_img, img, shift):
    diff_img = fixed_img - np.roll(img, [shift[0], shift[1]], axis=[0, 1])
    return (np.abs(np.diff(diff_img, axis=0, prepend=1))+np.abs(np.diff(diff_img, axis=1, prepend=1))).sum()

def tv_l2_pixel2(fixed_img, img, shift):
    diff_img = fixed_img - np.roll(img, [shift[0], shift[1]], axis=[0, 1])        
    return ((np.diff(diff_img, axis=0, prepend=1))**2+(np.diff(diff_img, axis=1, prepend=1))**2).sum() 

def multi_resolutin_tv_reg2(fixed_img, img, levs=4, wz=10, step=0.2):
    sch_config = {}
    sch_config[levs-1] = {'wz':20, 'step':step}
    # tv1_pxl = np.ndarray([levs, wz, wz], dtype=np.float32)
    tv1_pxl = {}
    tv1_pxl_id = np.zeros(levs, dtype=np.int16)
    shift = np.zeros([levs, 2], dtype=np.float32)
    rlt = []
    
    for ii in np.int32(np.arange(levs-1)):
        sch_config[levs-2-ii] = {'wz':wz, 'step':2**ii}
        
    # print(tv1_pxl.shape)
    
    n_cpu = os.cpu_count()
    
    for ii in range(levs-1):
        print(time.asctime())
        print(ii)
        w = sch_config[ii]['wz']
        step = int(sch_config[ii]['step'])
        print(0.1)
        
        chunksize = int(w**2/(n_cpu-1))
        with ProcessPool() as pool:
            future = pool.map(partial(tv_l1_pixel2, fixed_img, img), 
                              [[int(step*(jj//w-int(w/2))+shift[:ii, 0].sum()), 
                                int(step*(jj%w-int(w/2))+shift[:ii, 1].sum())] for jj in np.int32(np.arange(w**2))],
                              time_out=2)
            iterator = future.result()
            
            while True:
                try:
                    rlt.append(next(iterator))
                except StopIteration:
                    print('option 1')
                    break
                except TimeoutError as error:
                    print('option 2')
                    print('function took longer than %d seconds'%error.args[1])
                except ProcessExpired as error:
                    print('option 3')
                    print('%s. Exit code: %d'%(error, error.exitcode))
                except Exception as error:
                    print('option 4')
                    print('function raised %s'%error)
                    print(error.traceback)
                    
        # # # with mp.Pool(int(n_cpu/2)) as pool:
        # with mp.get_context().Pool(int(n_cpu-1)) as pool:
        #     rlt = pool.map_async(partial(tv_l1_pixel, fixed_img, img), 
        #                       [[int(step*(jj//w-int(w/2))+shift[:ii, 0].sum()), 
        #                         int(step*(jj%w-int(w/2))+shift[:ii, 1].sum())] for jj in np.int32(np.arange(w**2))])
        #     rlt.wait()
        print(0.2)
        pool.close()
        pool.join()
        
        print(0.3)
        tem = np.ndarray([w, w], dtype=np.float32)
        for kk in range(w**2):
            tem[kk//w, kk%w] = rlt[kk]
        # del(rlt)
        gc.collect()
        
        tv1_pxl[ii] = np.array(tem)
        tv1_pxl_id[ii] = tv1_pxl[ii].argmin()
        shift[ii, 0] = step*(tv1_pxl_id[ii]//w-int(w/2))
        shift[ii, 1] = step*(tv1_pxl_id[ii]%w-int(w/2))
        print(ii)
        
    # print(tv1_pxl.shape)
    
    
    print(levs-1)
    w = sch_config[levs-1]['wz']
    step = sch_config[levs-1]['step']
    # with mp.Pool(int(n_cpu/2)) as pool:
    with mp.get_context('spawn').Pool(int(n_cpu/2)) as pool:
        rlt = pool.starmap(tv_l1_subpixel2, [(fixed_img, img, step*(jj//w-int(w/2))+shift[:(levs-1), 0].sum(), 
                                              step*(jj%w-int(w/2))+shift[:(levs-1), 1].sum()) for jj in np.int32(np.arange(w**2))])
    pool.close() 
    pool.join()
    
    print(0.3)
    tem = np.ndarray([w, w], dtype=np.float32)
    for kk in range(w**2):
        tem[kk//w, kk%w] = rlt[kk]
    del(rlt)
    gc.collect()
    
    tv1_pxl[levs-1] = np.array(tem)
    tv1_pxl_id[levs-1] = tv1_pxl[levs-1].argmin()
    shift[levs-1, 0] = step*(tv1_pxl_id[levs-1]//w-int(w/2))
    shift[levs-1, 1] = step*(tv1_pxl_id[levs-1]%w-int(w/2))
    
    print(levs-1)
    # print(tv1_pxl.shape)
        
    return tv1_pxl, tv1_pxl_id, shift, shift.sum(axis=0)

In [8]:
print(time.asctime())
#for ii in range(5):
#    tv1_pxl, tv1_pxl_id, shift_list, shift = multi_resolutin_tv_reg(fixed_img, img, levs=4, wz=10, step=0.2)
tv1_pxl, tv1_pxl_id, shift_list, shift = multi_resolutin_tv_reg2(fixed_img, img, levs=5, wz=20, step=0.2)
print(time.asctime())

Sun Oct  4 16:37:25 2020
Sun Oct  4 16:37:25 2020
0
0.1
option 1
0.2
0.3
0
Sun Oct  4 16:37:57 2020
1
0.1
option 1
0.2
0.3
1
Sun Oct  4 16:38:26 2020
2
0.1
option 1
0.2
0.3
2
Sun Oct  4 16:38:55 2020
3
0.1
option 1
0.2
0.3
3
4


NameError: name 'tv_l1_subpixel2' is not defined